In [1]:
import numpy as np
import cvxpy as cp
import itertools as it
import matplotlib.pyplot as plt
from scipy import sparse
from pygsp import graphs, filters
from numpy.linalg import inv
import networkx as nx
import random as rand
from sklearn import linear_model as lm

%run SyntheticDataGenerator.py
%run DongsAlgorithm.py
%run Metrics.py
%run Scripts/GSPRegression.py

# For reproducing results 
seed = 0

ModuleNotFoundError: No module named 'pygsp'

In [ ]:
G = nx.erdos_renyi_graph(20, 0.2, seed = 7)
S1 = RandomSignal(G, 100, 0, .5, 0)
L_ER = nx.laplacian_matrix(G).toarray()
normL_ER = (L_ER.shape[0]/np.trace(L_ER))*L_ER
E = GL_SigRep(0.002, .9, S1, .001, 100)
ComputeMetrics(E, normL_ER, .01)

In [ ]:
num_obser = 20
num_trials = 5
dist_vec = ['uniform', 'normal', 'binomial']
param_vec = [['0,1'], ['0,.5', '0,.5'], ['1,.5']]
M = generate_trials(num_obser, num_trials, dist_vec, param_vec, seed)
b = np.ones(4)
S2 = RandomRegressorSignal(G, 0, .5, b, M, seed)

In [ ]:
# Optimal hyperparamerters for signal w/o regressors
E = GL_SigRep(0.002, .9, S2, .001, 100)
ComputeMetrics(E, normL_ER, .01)

In [ ]:
# Hyperparameters optimized by first optimizing a, and then optimizing b.
a = 0.001
b = 0.14040404040404042
Etemp = GL_SigRep(a, b, S2, .001, 100) 
ComputeMetrics(Etemp, normL_ER, .01)

In [ ]:
# Hyperparameters optimized by alternating optimization
a = 0.001106060606060606
b = 0.49949494949494955
Etemp = GL_SigRep(a, b, S2, .001, 100) 
ComputeMetrics(Etemp, normL_ER, .01)

In [ ]:
a = 0.0092
b = 1
Etemp = GSP_Regression(a, b, S2, M, .001, 100)[0]
ComputeMetrics(Etemp, normL_ER, .01)

In [ ]:
a = 0.001106060606060606
b = 0.49949494949494955
ratio = np.linspace(0.45, 0.55, 100)
LGs = [GL_SigRep(a, r, S2, .001, 100) for r in ratio]  
Metrics = [ComputeMetrics(e, normL_ER, .01) for e in LGs]
fmes = [i['F-Measure'] for i in Metrics]

In [ ]:
#---------------------------------------------------------------------------------------------------------------------
# Optimize a
#(.001, 1, 100) fmes = 0.42152466367713 a = .001
#(.0001, .0015, 100) fmes = 0.44554455445544555 a = 0.001

# Optimize b
#(.5, 1.5, 100) fmes = 0.4568527918781726 b = 0.5
#(.1, .9, 100) fmes = 0.5 b = 0.14040404040404042

# Optimize a
#(.0005, .0015, 100) fmes = .5 a = 0.001, b = 0.14040404040404042
#---------------------------------------------------------------------------------------------------------------------
# Optimize with alternating 

# b = 1
#(.001, 1, 100) fmes = 0.42152466367713 a = .001

# a = 0.001
#(.5, 1.5, 100) fmes = 0.4568527918781726, b = 0.5

# b = 0.5
#(0.0005, 0.0015, 100) fmes = 0.46391752577319584, a = 0.001106060606060606

# a = 0.001106060606060606 
#(0.45, 0.55, 100) fmes = 0.46391752577319584, b = 0.49949494949494955 
#---------------------------------------------------------------------------------------------------------------------
# Optimize Precision
#---------------------------------------------------------------------------------------------------------------------
for i in range(len(fmes)):
    print(i, fmes[i], ratio[i])

In [ ]:
a = 0.0092
b = 0.49949494949494955
ratio = np.linspace(0.01, 2.1, 100)
LGs = [GSP_Regression(a, r, S2, M, .001, 100)[0] for r in ratio]  
Metrics = [ComputeMetrics(e, normL_ER, .01) for e in LGs]
fmes = [i['F-Measure'] for i in Metrics]

In [ ]:
# b = 1
#(0.1, 1.1, 100) fmes = 0.26666666666666666, a = 0.1101010101010101, 0.22121212121212122 <- This is strange,
# Why did this work!
#(.001, .2, 100) # ERROR 'list' object has no attribute 'transpose' line 55
#(.009, .2, 100) # ERROR ""
# Error is an issue with how regression coefficents are being recoverd.
# Removing this and proceeding.

#(.01, 2, 100) ERROR: invalid index to scalar variable.
# For some reason the return statement when threshold was met was not working as intended.
#-------------------------------------------------------------------------------------------------------------------
# b = 1
# (.01, 2, 100) fmes = 0.4406779661016949, a = 0.01
# (0.05, .15, 100) Seems like a poor way to optimize
#-------------------------------------------------------------------------------------------------------------------
# a = 0.01, b = 1
# (.5, 1.5, 100) No good
# (.001, 10, 100) fmes = 0.42152466367713, a = 0.001
# (0.0001, 0.01, 100) fmes = 0.4761904761904762 a = 0.0092 
# (0.009, 0.0093, 100) fmes = 0.4761904761904762 a = 0.0092

# a = 0.0092
# (0.01, 2.1, 100) fmes = 0.4761904761904762 a = 1.0022222222222223
for i in range(len(fmes)):
    print(i, fmes[i], ratio[i])

In [ ]:
# For some reason only the first pass learned a laplacian.
for e in LGs:
    print(e.shape)

In [ ]:
def GSP_Regression_test(a, b, Signal, P, tol, max_itr):
    print("Begining GSP Regression:")
    Y = Signal
    print("Initial shape of Y " + str(Y.shape))
    R = np.zeros(Y.shape)
    print("Initial shape of R " + str(R.shape))
    I = np.matrix(np.identity(Signal.shape[0]))
    L = Problem1(a, b, Y)
    print("Initial shape of L " + str(L.shape))
    for i in range(1,max_itr):
        print("___________________________________________________________________")
        print(str(i) + " pass")
        temp = L
        print("Current shape of L is " + str(L.shape) + " Learing Y.")
        # Closed form solution given by Dong et al.
        Y = (inv(I + a*L))@(Signal - R)
        print("Current shape of Y is " + str(Y.shape) + " Learning R.")
        # Find new Regressor matrix
        R = Signal - Y
        print("Current shape of R is " + str(R.shape) + " Learning L.")
        # Find a new L given Y.
        L = Problem1(a, b, Y)
        print("Shape of learning L is " + str(L.shape) + " Finished iteration")
        # Check if we are within the tolerance
        if(np.all(abs(temp - L) < tol)):
                print("Tolerance met.")
                return np.asarray(L)
        print("Tolerance not met.")
    #b = (inv(P.transpose()@P)@P.transpose)@np.matrix.flatten(R, 'F')
    #return [np.asarray(L), Y, b]
    print("Max numeber of iterations reached.")
    return [np.asarray(L), Y]

In [ ]:
E = GSP_Regression_test2(.01, 1, S2, M, .001, 100)[0]

In [ ]:
E[1].shape

In [ ]:
E = GSP_Regression_test2(.01, 1, S2, M, .001, 100)[0]
ComputeMetrics(E, normL_ER, .01)

In [ ]:
E[0].shape

In [ ]:
def GSP_Regression_test2(a, b, Signal, P, tol, max_itr):
    print("Begining GSP Regression:")
    Y = Signal
    print("Initial shape of Y " + str(Y.shape))
    R = np.zeros(Y.shape)
    print("Initial shape of R " + str(R.shape))
    I = np.matrix(np.identity(Signal.shape[0]))
    L = Problem1(a, b, Y)
    print("Initial shape of L " + str(L.shape))
    for i in range(1,max_itr):
        print("___________________________________________________________________")
        print(str(i) + " pass")
        temp = L
        print("Current shape of L is " + str(L.shape) + " Learing Y.")
        # Closed form solution given by Dong et al.
        Y = (inv(I + a*L))@(Signal - R)
        print("Current shape of Y is " + str(Y.shape) + " Learning R.")
        # Find new Regressor matrix
        R = Signal - Y
        print("Current shape of R is " + str(R.shape) + " Learning L.")
        # Find a new L given Y.
        L = Problem1(a, b, Y)
        print("Shape of learning L is " + str(L.shape) + " Finished iteration")
        # Check if we are within the tolerance
        if(np.all(abs(temp - L) < tol)):
                print("Tolerance met.")
                break
        print("Tolerance not met.")
    #b = (inv(P.transpose()@P)@P.transpose)@np.matrix.flatten(R, 'F')
    #return [np.asarray(L), Y, b]
    print("Max numeber of iterations reached.")
    return [np.asarray(L), Y]